In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.conf import SparkConf
config = SparkConf()
# config.set("property", "value")
config.setMaster("local").setAppName("SparkHelloWorldStream")

from pyspark.sql import SparkSession
# spark Session, entry point for Spark SQL, DataFrame
spark = SparkSession.builder\
                    .config(conf=config)\
                    .getOrCreate()

sc = spark.sparkContext

In [3]:
# pre-requistes

# we need a server running on port 9999, that provides text stream to spark
# spark stream is a client, that will connect ot a server running on port 9999

# terminal, 

# nc -lk 9999

In [3]:
# Structured Streaming 
# based on Data Frame/ internally there is RDD
# Micro Batch, trigger is default one, as soon as possible when data arrives
# trigger time approx 100 ms to 1 sec
# spark batch programming, we use spark.read
# in streaming, we use readStream
# socket is predefined word, represent data comes from TCP/IP socket
# spark is client
lineStreamDf = spark\
                .readStream\
                .format("socket")\
                .option("host", "localhost")\
                .option("port", 9999)\
                .load()

# now spark shall connect the server running on port 9999
# then it will wait the input from server
# user/developer can write something on server console and Press Enter key
# to send to spark / client

In [4]:
# printSchema vary based on driver we use. for socket, we have only value
lineStreamDf.printSchema()
# show will not work for stream, it will cause error
#lineStreamDf.show() # good for batchs, not for stream

root
 |-- value: string (nullable = true)



In [5]:
# print data on console 
# for stream write, we should use writeStream
# for batch use write
# append is mode
# console is keyword, present console output
# writeStream is an action, which runs the whole show,
# reading data, trigger window etc by writeStream
# in general 1 sec trigger used if not mentioned, default
echoOnconsole = lineStreamDf\
                .writeStream\
                .outputMode("append")\
                .format("console")\
                .start()

# start basically start the streaming, listen for input
# as input arrives, it starts processing
# listen for jupyter console for the output

Py4JJavaError: An error occurred while calling o46.start.
: java.lang.UnsupportedOperationException: Data source socket does not support streamed writing
	at org.apache.spark.sql.execution.datasources.DataSource.createSink(DataSource.scala:335)
	at org.apache.spark.sql.streaming.DataStreamWriter.createV1Sink(DataStreamWriter.scala:484)
	at org.apache.spark.sql.streaming.DataStreamWriter.startInternal(DataStreamWriter.scala:450)
	at org.apache.spark.sql.streaming.DataStreamWriter.start(DataStreamWriter.scala:301)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.comman